여기에 블렌딩 관련 작업

In [1]:
# from laptop w/o gpu
import numpy as np
import cv2
import time
import os
from enum import Enum
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
from utils import *

print(cv2.__version__)

4.11.0


# 경계 블렌딩

우선 아래 방법 적용하지 않은 상태로 구현

- 원본 파트에서 경계에서 떨어진 거리에 따라 블렌딩 비율 다르게
- 경계에 가까울수록 downscaled->upscaled 이미지 비율을 높게 잡고
- 경계에서 멀어질수록 original part 이미지 비율을 높게 설정하여 블렌딩
- 일정 거리 이상으로 멀어지면 100% 원본파트
- 선형, sigmoid 등 함수로 interpolation

In [2]:
class blend_mode(Enum):
    alpha = 0
    poisson = 1
    laplacian = 2
    grad = 3
    dl = 4

def blending_two_images(src, target, mask, mode = blend_mode.poisson, center = (0, 0)):
    match mode:
        case blend_mode.alpha:
            result = cv2.addWeighted(src, 0.7, target, 0.3, 0, mask=mask)
        case blend_mode.poisson:
            result = cv2.seamlessClone(
                src, target, mask, center, cv2.MIXED_CLONE
            )
        case blend_mode.laplacian:
            # laplacian dist
            return
        case blend_mode.grad:
            # gradient
            return
        case blend_mode.dl:
            # dl model
            return

    return result

In [5]:
img_path = 'Lenna_(test_image).png'

# down->upscaled 된거 path
# original part path
